In [21]:
from omnifold import DataLoader, MLP, MultiFold 
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm import tqdm  # Add progress bars

In [2]:
data = np.load('rawdata_omnifold.npz')
ndim = 6

In [3]:
observables = ["m", "M", "w", "tau21", "zg", "sdm"]

true, reco, true_alt, reco_alt = (
    np.column_stack([data[f"{obs}_{suffix}"] for obs in observables])
    for suffix in ["true", "reco", "true_alt", "reco_alt"]
)
true_train, true_test, reco_train, reco_test,  = train_test_split(
    true, reco, test_size=0.25, random_state=42
)

true_alt_train, true_alt_test, reco_alt_train, reco_alt_test = train_test_split(
    true_alt, reco_alt, test_size=0.25, random_state=42
)


In [6]:
nature_loader = DataLoader(
            reco = reco_train,
            gen = true_train,
            bootstrap=False,
            normalize=True)
mc_loader = DataLoader(
            reco = reco_alt_train,
            gen = true_alt_train,
            bootstrap=False,
            normalize=True)

INFO: Creating weights ...
INFO: Creating pass reco flag ...
INFO: Creating pass gen flag ...
INFO: Normalizing sum of weights to 1000000 ...
INFO: Creating weights ...
INFO: Creating pass reco flag ...
INFO: Creating pass gen flag ...
INFO: Normalizing sum of weights to 1000000 ...


In [7]:
ndim = 6
reco_model = MLP(ndim, layer_sizes = [128,128,128])
gen_model = MLP(ndim, [128,128,128])

omnifold = MultiFold(
    "RAN Comparison",
    reco_model,
    gen_model,
    nature_loader,
    mc_loader,
    batch_size = 512,
    niter = 5,
    epochs = 20,
    verbose = True,
    lr = 5e-5,
)
omnifold.Unfold()
unfolded_weights  = omnifold.reweight(true_alt_test,omnifold.model2,batch_size=1000)

W0000 00:00:1741966760.327901 2857293 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


4862 training steps at reco and 4942 steps at gen
ITERATION: 1
RUNNING STEP 1
Creating cached data from step 1
################################################################################
Train events used: 2489344, Test events used: 497868
################################################################################
Epoch 1/20


KeyboardInterrupt: 

In [6]:
unfolded_weights  = omnifold.reweight(true_alt,omnifold.model2,batch_size=1000)
ran_weights = np.random.normal(np.ones_like(data['m_true']),20)

  48/1687 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step

1687/1687 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step


In [7]:
np.savez('weights_omnifold.npz', unfolded_weights=unfolded_weights, ran_weights=ran_weights)

In [13]:
data = np.load('rawdata_omnifold.npz')
weights = np.load('weights_omnifold.npz')
substructure_variables = ['m', 'M', 'w', 'tau21', 'zg', 'sdm']

# a dictionary to hold information about the observables
obs = {}

# the jet mass and histogram style information
obs.setdefault('m', {}).update({
    'xlim': (0, 75), 'ylim': (0, 0.065),
    'xlabel': r'Jet Mass', 'symbol': r'$m$ [GeV]',
    'stamp_xy': (0.425, 0.65),
})

# the constituent multiplicity and histogram style information
obs.setdefault('M', {}).update({
    'xlim': (0, 80), 'ylim': (0, 0.065),
    'xlabel': 'Jet Constituent Multiplicity', 'symbol': r'$M$',
    'stamp_xy': (0.42, 0.65),
})

# the jet width and histogram style information
obs.setdefault('w', {}).update({
    'xlim': (0, 0.6), 'ylim': (0, 10),
    'xlabel': r'Jet Width', 'symbol': r'$w$',
    'stamp_xy': (0.425, 0.65),
})

# the N-subjettiness ratio and histogram style information
obs.setdefault('tau21', {}).update({
    'xlim': (0, 1.2), 'ylim': (0, 3),
    'xlabel': r'$N$-subjettiness Ratio', 'symbol': r'$\tau_{21}^{(\beta=1)}$',
    'stamp_xy': (0.41, 0.92),
    'legend_loc': 'upper left', 'legend_ncol': 1,
})

# the groomed momentum fraction and histogram style information
obs.setdefault('zg', {}).update({
    'xlim': (0, 0.5), 'ylim': (0, 9),
    'xlabel': r'Groomed Jet Momentum Fraction', 'symbol': r'$z_g$',
    'stamp_xy': (0.425, 0.65),
})

# the groomed jet mass and histogram style information
obs.setdefault('sdm', {}).update({
    'xlim': (-14, -2), 'ylim': (0, 0.3),
    'xlabel': r'Soft Drop Jet Mass', 'symbol': r'$\ln\rho$',
    'stamp_xy': (0.41, 0.92),
    'legend_loc': 'upper left', 'legend_ncol': 1,
})

In [ ]:
'FUNCTIONS'
normalize = lambda x: x / np.sum(x, axis=0)

def generate_purity_bins(truth, data, min_purity=0.5, epsilon=1e-6, max_bins_per_dim=15):
    """
    Generate adaptive bins for each dimension based on purity, using predefined min/max limits
    
    Parameters:
    - truth: Truth data with shape (ndim, N)
    - data: Reconstructed data with shape (ndim, N)
    - min_purity: Minimum purity required for a bin
    - epsilon: Small value to avoid numerical issues
    - max_bins_per_dim: Maximum number of bins per dimension
    
    Returns:
    - List of bin arrays for each dimension
    """
    ndim = truth.shape[0]
    all_bins = []
    
    # Map dimension indices to variable names
    dim_to_var = {
        0: 'm',     # Jet mass
        1: 'M',     # Multiplicity 
        2: 'w',     # Width
        3: 'tau21', # N-subjettiness
        4: 'zg',    # Groomed momentum fraction
        5: 'sdm',   # Soft Drop Mass
    }
    
    for dim in range(ndim):
        # Get variable name for this dimension
        var_name = dim_to_var.get(dim)
        
        if var_name and var_name in obs:
            # Use predefined min/max from the obs dictionary
            min_val, max_val = obs[var_name]['xlim']
            print(f"Dimension {dim} ({var_name}): Using predefined range [{min_val}, {max_val}]")
        else:
            # Fall back to data min/max if no predefined limits
            min_val = truth[dim].min()
            max_val = truth[dim].max()
            print(f"Dimension {dim}: Using data range [{min_val:.4f}, {max_val:.4f}]")
        
        # Start with the minimum value
        bins = [min_val]
        i = 0
        
        # Create adaptive bins based on purity
        while bins[-1] < max_val and i < len(bins) and len(bins) < max_bins_per_dim:
            for binhigh in np.linspace(bins[i] + epsilon, max_val, 20):
                in_bin = (truth[dim] > bins[i]) & (truth[dim] < binhigh)
                in_reco_bin = (data[dim] > bins[i]) & (data[dim] < binhigh)
                if np.sum(in_bin) > 0:
                    purity = np.sum(in_bin & in_reco_bin) / np.sum(in_bin)
                    if purity > min_purity:
                        i += 1
                        bins.append(binhigh)
                        break
            else:
                break
        
        # Ensure the last bin includes the maximum value
        if bins[-1] < max_val:
            bins.append(max_val)
            
        all_bins.append(np.array(bins))
    
    # Print info about number of bins per dimension
    print("\nBins per dimension:")
    for dim, bins in enumerate(all_bins):
        var_name = dim_to_var.get(dim, f"Dimension {dim}")
        print(f"  {var_name}: {len(bins)-1} bins, range: [{bins[0]:.4f}, {bins[-1]:.4f}]")
    
    # Calculate total size of the full matrix
    total_bins = 1
    for bins in all_bins:
        total_bins *= (len(bins)-1)
    print(f"Total bins in truth space: {total_bins}")
    
    return all_bins

def create_response_matrix_nd(gen, sim, bins):
    """
    Create N-dimensional response matrix.
    
    Parameters:
    - gen: Truth data with shape (ndim, N)
    - sim: Simulated/reconstructed data with shape (ndim, N)
    - bins: List of bin edges for each dimension
    
    Returns:
    - Response matrix mapping from truth to measured space
    """
    ndim = gen.shape[0]
    
    # Create a list of bins for all 2*ndim dimensions
    # First ndim dimensions are for gen, last ndim for sim
    bins_list = []
    for i in range(ndim):
        bins_list.append(bins[i])  # Add gen bins
    for i in range(ndim):
        bins_list.append(bins[i])  # Add sim bins
    
    # Stack data for histogramdd: [gen_dim1, gen_dim2, ..., sim_dim1, sim_dim2, ...]
    combined_data = np.vstack([gen, sim]).T
    
    # Create the joint histogram
    H, _ = np.histogramdd(combined_data, bins=bins_list)
    
    # Normalize to get conditional probabilities
    # Sum over the sim dimensions (last ndim dimensions)
    sum_axes = tuple(range(ndim, 2*ndim))
    norm = np.sum(H, axis=sum_axes, keepdims=True)
    H_norm = np.divide(H, norm, where=norm>0, out=np.zeros_like(H, dtype=float))
    H_norm[np.isnan(H_norm)] = 0
    
    return H_norm


def bayesian_unfolding_step_nd(R, f, data_hist):
    """
    Perform one step of Bayesian unfolding for N-dimensional data.
    
    Parameters:
    - R: Response matrix from create_response_matrix_nd
    - f: Current estimate of true distribution
    - data_hist: Measured data histogram
    
    Returns:
    - Updated estimate of true distribution
    """
    ndim = len(data_hist.shape)
    
    # Expected measurements: R applied to f
    axes = [list(range(ndim, 2*ndim)), list(range(ndim))]
    expected = np.tensordot(R, f, axes=axes)
    
    # Reweighting factors
    reweight = np.divide(data_hist, expected, where=expected>0, out=np.zeros_like(data_hist, dtype=float))
    reweight[np.isnan(reweight)] = 0
    
    # Apply reweighting to get updated f
    axes = [list(range(ndim)), list(range(ndim))]
    f_prime = f * np.tensordot(R, reweight, axes=axes)
    
    return f_prime

def iterative_bayesian_unfolding_nd(data, gen, sim, bins, n_iterations):
    """
    Perform iterative Bayesian unfolding for N-dimensional data.
    
    Parameters:
    - data: Measured data with shape (ndim, N)
    - gen: Generated/truth data with shape (ndim, N)
    - sim: Simulated/reconstructed data with shape (ndim, N)
    - bins: Bin specification for all dimensions
    - n_iterations: Number of unfolding iterations
    
    Returns:
    - List of unfolded distributions for each iteration
    """    
    # Create the response matrix
    R = create_response_matrix_nd(gen, sim, bins)
    
    # Initial histogram
    f, _ = np.histogramdd(gen.T, bins=bins)
    data_hist, _ = np.histogramdd(data.T, bins=bins)
    
    for i in range(n_iterations):
        f = bayesian_unfolding_step_nd(R, f, data_hist)
    
    return f

def plot_unfolding_results(true_hist, unfolded_hist, bins, dim_to_plot=0, fig=None, ax=None):
    """
    Plot unfolding results for a specified dimension by integrating over other dimensions
    
    Parameters:
    - true_hist: True histogram
    - unfolded_hist: Unfolded histogram
    - bins: Bin edges used for histograms
    - dim_to_plot: Which dimension to plot (0-based index)
    """
    ndim = len(bins)
    
    # Sum over all dimensions except the one to plot
    sum_axes = tuple([i for i in range(ndim) if i != dim_to_plot])
    true_hist_1d = np.sum(true_hist, axis=sum_axes)
    unfolded_hist_1d = np.sum(unfolded_hist, axis=sum_axes)
    
    # Normalize histograms
    true_hist_1d = true_hist_1d / np.sum(true_hist_1d)
    unfolded_hist_1d = unfolded_hist_1d / np.sum(unfolded_hist_1d)
    
    # Create bin centers for plotting
    bin_centers = 0.5 * (bins[dim_to_plot][1:] + bins[dim_to_plot][:-1])
    bin_widths = np.diff(bins[dim_to_plot])
    
    # Create figure if not provided
    if fig is None or ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot histograms
    ax.bar(bin_centers, true_hist_1d, width=bin_widths, alpha=0.5, label='True')
    ax.bar(bin_centers, unfolded_hist_1d, width=bin_widths, alpha=0.5, label='Unfolded')
    
    ax.set_xlabel(f'Dimension {dim_to_plot}')
    ax.set_ylabel('Normalized Counts')
    ax.legend()
    
    return fig, ax


In [15]:
bins = generate_purity_bins(true_train.T, reco_train.T, min_purity=(0.25))

Dimension 0 (m): Using predefined range [0, 75]
Dimension 1 (M): Using predefined range [0, 80]
Dimension 2 (w): Using predefined range [0, 0.6]
Dimension 3 (tau21): Using predefined range [0, 1.2]
Dimension 4 (zg): Using predefined range [0, 0.5]
Dimension 5 (sdm): Using predefined range [-14, -2]

Bins per dimension:
  m: 15 bins, range: [0.0000, 75.0000]
  M: 7 bins, range: [0.0000, 80.0000]
  w: 15 bins, range: [0.0000, 0.6000]
  tau21: 14 bins, range: [0.0000, 1.2000]
  zg: 9 bins, range: [0.0000, 0.5000]
  sdm: 15 bins, range: [-14.0000, -2.0000]
Total bins in truth space: 2976750


In [18]:
unfolded_hists = iterative_bayesian_unfolding_nd(reco_train.T, true_alt_train.T, reco_alt_train.T, bins, 5)

MemoryError: Unable to allocate 441. TiB for an array with shape (60562512324864,) and data type int64

In [16]:
import pickle

# Save everything
with open('IBU_omnifold.pkl', 'wb') as f:
    pickle.dump({
        'bins': bins
    }, f)

In [19]:
reco_train.shape

(1224331, 6)

In [25]:
def create_sparse_response_matrix(gen, sim, bins):
    """
    Create a memory-efficient sparse representation of the response matrix
    
    Parameters:
    - gen: Truth data (can be either shape (N, ndim) or (ndim, N))
    - sim: Simulated/reconstructed data (same shape as gen)
    - bins: List of bin edges for each dimension
    
    Returns:
    - Dictionary mapping (gen_idx, sim_idx) to normalized response probability
    """
    # Check and fix data orientation
    if gen.shape[0] > gen.shape[1] and len(bins) == gen.shape[1]:
        # Data is in shape (N, ndim), so transpose to (ndim, N)
        print(f"Transposing data from shape {gen.shape} to {gen.shape[::-1]}")
        gen = gen.T
        sim = sim.T
    
    ndim = gen.shape[0]
    n_events = gen.shape[1]
    
    # Verify dimensions match
    if len(bins) != ndim:
        raise ValueError(f"Number of bin arrays ({len(bins)}) doesn't match data dimensions ({ndim})")
    
    print(f"Creating sparse response matrix for {ndim}D data with {n_events} events")
    print(f"Bins shape: {[len(b) for b in bins]}")
    
    # Create dictionaries to store counts
    joint_counts = defaultdict(int)
    gen_counts = defaultdict(int)
    
    # Process events in batches to avoid memory issues
    batch_size = 10000
    for batch_start in tqdm(range(0, n_events, batch_size), desc="Processing batches"):
        batch_end = min(batch_start + batch_size, n_events)
        
        # Calculate bin indices for this batch
        gen_indices = []
        sim_indices = []
        
        for dim in range(ndim):
            gen_idx = np.digitize(gen[dim, batch_start:batch_end], bins[dim]) - 1
            sim_idx = np.digitize(sim[dim, batch_start:batch_end], bins[dim]) - 1
            
            # Clip indices to valid range
            gen_idx = np.clip(gen_idx, 0, len(bins[dim])-2)
            sim_idx = np.clip(sim_idx, 0, len(bins[dim])-2)
            
            gen_indices.append(gen_idx)
            sim_indices.append(sim_idx)
        
        # Count events in each (gen, sim) bin combination
        for i in range(batch_end - batch_start):
            gen_idx = tuple(gen_indices[d][i] for d in range(ndim))
            sim_idx = tuple(sim_indices[d][i] for d in range(ndim))
            
            joint_counts[(gen_idx, sim_idx)] += 1
            gen_counts[gen_idx] += 1
    
    # Normalize to get conditional probabilities P(sim|gen)
    print("Normalizing response matrix...")
    response_dict = {}
    for (gen_idx, sim_idx), count in joint_counts.items():
        gen_count = gen_counts[gen_idx]
        if gen_count > 0:
            response_dict[(gen_idx, sim_idx)] = count / gen_count
    
    print(f"Response matrix size: {len(response_dict)} non-zero elements")
    return response_dict, gen_counts
def create_histogram_nd(data, bins):
    """Create N-dimensional histogram using batch processing"""
    # Check and fix data orientation
    if data.shape[0] > data.shape[1] and len(bins) == data.shape[1]:
        # Data is in shape (N, ndim), so transpose to (ndim, N)
        print(f"Transposing data from shape {data.shape} to {data.shape[::-1]}")
        data = data.T
    
    ndim = data.shape[0]
    n_events = data.shape[1]
    
    # Verify dimensions match
    if len(bins) != ndim:
        raise ValueError(f"Number of bin arrays ({len(bins)}) doesn't match data dimensions ({ndim})")
    
    # Initialize dict for histogram counts
    hist_dict = defaultdict(int)
    
    # Process in batches
    batch_size = 10000
    for batch_start in tqdm(range(0, n_events, batch_size), desc="Creating histogram"):
        batch_end = min(batch_start + batch_size, n_events)
        
        # Calculate bin indices for each dimension
        indices = []
        for dim in range(ndim):
            idx = np.digitize(data[dim, batch_start:batch_end], bins[dim]) - 1
            idx = np.clip(idx, 0, len(bins[dim])-2)
            indices.append(idx)
        
        # Increment counts
        for i in range(batch_end - batch_start):
            bin_idx = tuple(indices[d][i] for d in range(ndim))
            hist_dict[bin_idx] += 1
    
    return hist_dict

In [26]:
def iterative_bayesian_unfolding_sparse(data, gen, sim, bins, n_iterations):
    """
    Perform IBU with sparse matrices for memory efficiency
    
    Parameters:
    - data: Measured data to unfold
    - gen: Generated/truth training data
    - sim: Simulated/reconstructed training data 
    - bins: List of bin edges for each dimension
    - n_iterations: Number of unfolding iterations
    
    Returns:
    - Dense array of unfolded distribution after final iteration
    """
    # Print shapes to debug
    print(f"Data shape: {data.shape}")
    print(f"Gen shape: {gen.shape}")
    print(f"Sim shape: {sim.shape}")
    print(f"Number of bin arrays: {len(bins)}")
    
    # Create sparse response matrix
    response_dict, gen_counts = create_sparse_response_matrix(gen, sim, bins)
    
    # Create initial histograms
    print("Creating initial histograms...")
    f_dict = create_histogram_nd(gen, bins)
    data_dict = create_histogram_nd(data, bins)
    
    # Perform iterative unfolding
    print("Starting iterative unfolding...")
    for iter_idx in range(n_iterations):
        print(f"Iteration {iter_idx+1}/{n_iterations}")
        f_dict = bayesian_unfolding_step_sparse(response_dict, gen_counts, f_dict, data_dict)
    
    # Convert final result to dense array
    print("Converting result to dense array...")
    result = sparse_to_dense(f_dict, bins)
    
    return result

In [32]:
# Print shapes to debug before calling
print(f"reco_test shape: {reco_test.shape}")
print(f"true_train shape: {true_train.shape}")
print(f"reco_train shape: {reco_train.shape}")
print(f"Number of bin arrays: {len(bins)}")

# Run the unfolding (will handle transposition if needed)
unfolded_hist = iterative_bayesian_unfolding_sparse(
    reco_test,
    true_alt_train,
    reco_alt_train,
    bins,
    n_iterations=5
)

# Create true test histogram for comparison
true_test_hist = sparse_to_dense(create_histogram_nd(true_test, bins), bins)

reco_test shape: (408111, 6)
true_train shape: (1224331, 6)
reco_train shape: (1224331, 6)
Number of bin arrays: 6
Data shape: (408111, 6)
Gen shape: (1265217, 6)
Sim shape: (1265217, 6)
Number of bin arrays: 6
Transposing data from shape (1265217, 6) to (6, 1265217)
Creating sparse response matrix for 6D data with 1265217 events
Bins shape: [16, 8, 16, 15, 10, 16]


Processing batches: 100%|██████████| 127/127 [00:06<00:00, 20.42it/s]


Normalizing response matrix...
Response matrix size: 1157966 non-zero elements
Creating initial histograms...
Transposing data from shape (1265217, 6) to (6, 1265217)


Creating histogram: 100%|██████████| 127/127 [00:02<00:00, 51.00it/s]


Transposing data from shape (408111, 6) to (6, 408111)


Creating histogram: 100%|██████████| 41/41 [00:00<00:00, 51.54it/s]


Starting iterative unfolding...
Iteration 1/5
Iteration 2/5
Iteration 3/5
Iteration 4/5
Iteration 5/5
Converting result to dense array...
Transposing data from shape (408111, 6) to (6, 408111)


Creating histogram: 100%|██████████| 41/41 [00:00<00:00, 51.64it/s]


In [34]:
with open('IBU_omnifold.pkl', 'wb') as f:
    pickle.dump({
        'unfolded_hist': unfolded_hist,  # Make sure this key exists!
        'true_test_hist': true_test_hist,
        'bins': bins
    }, f)